In [9]:
import numpy as np
import cv2 as cv
import glob
import yaml
from pathlib import Path

CHESSBOARD_CORNER_NUM_X = 10
CHESSBOARD_CORNER_NUM_Y = 7
IMAGE_SRC = "cam1_images"
CAMERA_PARAMETERS_OUTPUT_FILE = "cam1.yaml"

root = Path.cwd()

# Set path to the images
calib_imgs_path = root.joinpath(IMAGE_SRC)

# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(8,5,0)
objp = np.zeros((CHESSBOARD_CORNER_NUM_X*CHESSBOARD_CORNER_NUM_Y, 3), np.float32)
objp[:, :2] = np.mgrid[0:CHESSBOARD_CORNER_NUM_X, 0:CHESSBOARD_CORNER_NUM_Y].T.reshape(-1, 2)

# Arrays to store object points and image points from all the images.
objpoints = []  # 3d point in real world space
imgpoints = []  # 2d points in image plane.

images = calib_imgs_path.glob('*.jpg')
for fname in images:
    img = cv.imread(str(root.joinpath(fname)))
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, (CHESSBOARD_CORNER_NUM_X, CHESSBOARD_CORNER_NUM_Y), None)
    # If found, add object points, image points (after refining them)
    if ret:
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
        imgpoints.append(corners)
        # Draw and display the corners
        cv.drawChessboardCorners(img, (CHESSBOARD_CORNER_NUM_X, CHESSBOARD_CORNER_NUM_Y), corners2, ret)
        cv.imshow('img', img)
        cv.waitKey(500)
    else:
        print('Failed to find a chessboard in {}'.format(fname))

cv.destroyAllWindows()

print("Calibrating camera .... Please wait...")
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

print("Camera matrix is \n", mtx, "\n And is stored in calibration.yaml file along with distortion coefficients : \n", dist)

mean_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv.norm(imgpoints[i], imgpoints2, cv.NORM_L2) / len(imgpoints2)
    mean_error += error

print("Total error: {}".format(mean_error / len(objpoints)))

data = {'camera_matrix': np.asarray(mtx).tolist(), 'dist_coeff': np.asarray(dist).tolist()}
with open(CAMERA_PARAMETERS_OUTPUT_FILE, "w") as f:
    yaml.dump(data, f)


Calibrating camera .... Please wait...
Camera matrix is 
 [[901.44184954   0.         627.54487483]
 [  0.         904.66841265 379.96880038]
 [  0.           0.           1.        ]] 
 And is stored in calibration.yaml file along with distortion coefficients : 
 [[ 2.90064989e-01 -1.34252056e+00 -1.08080778e-03 -1.91921295e-03
   2.35356968e+00]]
Total error: 0.05515785854432075


# USING CHESSBOARD

In [5]:
import numpy as np
import cv2 as cv
import glob
import yaml
from pathlib import Path

CHESSBOARD_CORNER_NUM_X = 10
CHESSBOARD_CORNER_NUM_Y = 7
IMAGE_SRC = "cam1_images"
CAMERA_PARAMETERS_OUTPUT_FILE = "cam1.yaml"

# Thay vì sử dụng __file__, bạn sử dụng thư mục hiện tại
root = Path.cwd()

# Set path to the images
calib_imgs_path = root.joinpath(IMAGE_SRC)

# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
square_size = 24  # Giả sử độ dài cạnh của mỗi ô vuông là 25mm

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(9,6,0) và nhân với kích thước ô vuông thực tế
objp = np.zeros((CHESSBOARD_CORNER_NUM_X*CHESSBOARD_CORNER_NUM_Y, 3), np.float32)
objp[:, :2] = np.mgrid[0:CHESSBOARD_CORNER_NUM_X, 0:CHESSBOARD_CORNER_NUM_Y].T.reshape(-1, 2) * square_size

# Arrays to store object points and image points from all the images.
objpoints = []  # 3d point in real world space
imgpoints = []  # 2d points in image plane.

images = calib_imgs_path.glob('*.jpg')
for fname in images:
    img = cv.imread(str(root.joinpath(fname)))
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, (CHESSBOARD_CORNER_NUM_X, CHESSBOARD_CORNER_NUM_Y), None)
    # If found, add object points, image points (after refining them)
    if ret:
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
        imgpoints.append(corners)
        # Draw and display the corners
        cv.drawChessboardCorners(img, (CHESSBOARD_CORNER_NUM_X, CHESSBOARD_CORNER_NUM_Y), corners2, ret)
        cv.imshow('img', img)
        cv.waitKey(500)
    else:
        print('Failed to find a chessboard in {}'.format(fname))

cv.destroyAllWindows()

print("Calibrating camera .... Please wait...")
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

print("Camera matrix is \n", mtx, "\n And is stored in calibration.yaml file along with distortion coefficients : \n", dist)

mean_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv.norm(imgpoints[i], imgpoints2, cv.NORM_L2) / len(imgpoints2)
    mean_error += error

print("Total error: {}".format(mean_error / len(objpoints)))

data = {'camera_matrix': np.asarray(mtx).tolist(), 'dist_coeff': np.asarray(dist).tolist()}
with open(CAMERA_PARAMETERS_OUTPUT_FILE, "w") as f:
    yaml.dump(data, f)


Calibrating camera .... Please wait...
Camera matrix is 
 [[901.44204508   0.         627.54480678]
 [  0.         904.66861291 379.96878104]
 [  0.           0.           1.        ]] 
 And is stored in calibration.yaml file along with distortion coefficients : 
 [[ 2.90064547e-01 -1.34251639e+00 -1.08082547e-03 -1.91922978e-03
   2.35356175e+00]]
Total error: 0.055157898976098005


In [1]:
"""
This code assumes that images used for calibration are of the same arUco marker board provided with code

"""

import cv2
from cv2 import aruco
import yaml
import numpy as np
from pathlib import Path
from tqdm import tqdm

# root directory of repo for relative path specification.
root = Path.cwd()

# Set this flsg True for calibrating camera and False for validating results real time
calibrate_camera = True

# Set path to the images
calib_imgs_path = root.joinpath("C:\\Users\\phamnhat\\Desktop\\computer vision\\cam2_images")

# For validating results, show aruco board to camera.
aruco_dict = aruco.getPredefinedDictionary( aruco.DICT_6X6_1000 )

#Provide length of the marker's side
markerLength = 3.75  # Here, measurement unit is centimetre.

# Provide separation between markers
markerSeparation = 0.5   # Here, measurement unit is centimetre.

# create arUco board
board = aruco.GridBoard((4, 5), markerLength, markerSeparation, aruco_dict)




In [5]:

arucoParams = aruco.DetectorParameters()

if calibrate_camera == True:
    img_list = []
    calib_fnms = calib_imgs_path.glob('*.jpg')
    print('Using ...', end='')
    for idx, fn in enumerate(calib_fnms):
        print(idx, '', end='')
        img = cv2.imread( str(root.joinpath(fn) ))
        img_list.append( img )
        h, w, c = img.shape
    print('Calibration images')

    counter, corners_list, id_list = [], [], []
    first = True
    for im in tqdm(img_list):
        img_gray = cv2.cvtColor(im,cv2.COLOR_RGB2GRAY)
        corners, ids, rejectedImgPoints = aruco.detectMarkers(img_gray, aruco_dict, parameters=arucoParams)
        if first == True:
            corners_list = corners
            id_list = ids
            first = False
        else:
            corners_list = np.vstack((corners_list, corners))
            id_list = np.vstack((id_list,ids))
        counter.append(len(ids))
    print('Found {} unique markers'.format(np.unique(ids)))

    counter = np.array(counter)
    print ("Calibrating camera .... Please wait...")
    #mat = np.zeros((3,3), float)
    ret, mtx, dist, rvecs, tvecs = aruco.calibrateCameraAruco(corners_list, id_list, counter, board, img_gray.shape, None, None )

    print("Camera matrix is \n", mtx, "\n And is stored in calibration.yaml file along with distortion coefficients : \n", dist)
    data = {'camera_matrix': np.asarray(mtx).tolist(), 'dist_coeff': np.asarray(dist).tolist()}
    with open("calibration.yaml", "w") as f:
        yaml.dump(data, f)

else:
    camera = cv2.VideoCapture(0)
    ret, img = camera.read()

    with open('calibration.yaml') as f:
        loadeddict = yaml.load(f)
    mtx = loadeddict.get('camera_matrix')
    dist = loadeddict.get('dist_coeff')
    mtx = np.array(mtx)
    dist = np.array(dist)

    ret, img = camera.read()
    img_gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    h,  w = img_gray.shape[:2]
    newcameramtx, roi=cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))

    pose_r, pose_t = [], []
    while True:
        ret, img = camera.read()
        img_aruco = img
        im_gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
        h,  w = im_gray.shape[:2]
        dst = cv2.undistort(im_gray, mtx, dist, None, newcameramtx)
        corners, ids, rejectedImgPoints = aruco.detectMarkers(dst, aruco_dict, parameters=arucoParams)
        #cv2.imshow("original", img_gray)
        if corners == None:
            print ("pass")
        else:

            ret, rvec, tvec = aruco.estimatePoseBoard(corners, ids, board, newcameramtx, dist) # For a board
            print ("Rotation ", rvec, "Translation", tvec)
            if ret != 0:
                img_aruco = aruco.drawDetectedMarkers(img, corners, ids, (0,255,0))
                img_aruco = aruco.drawAxis(img_aruco, newcameramtx, dist, rvec, tvec, 10)    # axis length 100 can be changed according to your requirement

            if cv2.waitKey(0) & 0xFF == ord('q'):
                break;
        cv2.imshow("World co-ordinate frame axes", img_aruco)

cv2.destroyAllWindows()

Using ...0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 Calibration images


100%|██████████| 26/26 [00:00<00:00, 36.98it/s]


Found [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19] unique markers
Calibrating camera .... Please wait...
Camera matrix is 
 [[1.12827978e+03 0.00000000e+00 6.57164745e+02]
 [0.00000000e+00 1.13306387e+03 3.36362350e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]] 
 And is stored in calibration.yaml file along with distortion coefficients : 
 [[ 0.07304058 -0.79128477 -0.00191665  0.00451977  0.7964234 ]]
